In [41]:
import numpy as np
import tensorflow as tf

In [42]:
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [43]:
def initialize_parameters(n_a, n_x, n_y):
    Wax = np.random.randn(n_a, n_x)*0.01 
    Waa = np.random.randn(n_a, n_a)*0.01 
    Wya = np.random.randn(n_y, n_a)*0.01 
    b = np.zeros((n_a, 1)) 
    by = np.zeros((n_y, 1)) 
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

In [44]:
def rnn_step_forward(parameters, a_prev, x):
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.matmul(Wax, x) + np.matmul(Waa, a_prev) + b) 
    p_t = softmax(np.matmul(Wya, a_next) + by) 
    
    return a_next, p_t

In [45]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    gradients['dWya'] += np.matmul(dy, a.T)
    gradients['dby'] += dy
    da = np.matmul(parameters['Wya'].T, dy) + gradients['da_next'] 
    daraw = (1 - a * a) * da 
    gradients['db'] += daraw
    gradients['dWax'] += np.matmul(daraw, x.T)
    gradients['dWaa'] += np.matmul(daraw, a_prev.T)
    gradients['da_next'] = np.matmul(parameters['Waa'].T, daraw)
    return gradients

In [46]:
def update_parameters(parameters, gradients, lr):
    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

In [67]:
def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    loss = 0
    
    for t in range(len(X)):
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

In [68]:
def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a

In [69]:
def sample(parameters, char_to_ix):
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    x = np.zeros([len(char_to_ix),1])
    a_prev =np.zeros([n_a,1])
    
    indices = []
    
    idx = -1 
    
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        a = np.tanh(np.matmul(Wax,x)+np.matmul(Waa,a_prev)+b)
        z = np.matmul(Wya,a)+by
        y = softmax(z)
 
        idx = np.random.choice(len(char_to_ix),p=y[:,0])

        indices.append(idx)
        
        x = np.zeros([len(char_to_ix),1])
        x[idx] = 1
        
        a_prev = a
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [70]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    gradients, a = rnn_backward(X, Y, parameters, cache)
        
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]

In [71]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))

In [72]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [73]:
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]
    

In [74]:
def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

In [75]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):

    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    for j in range(num_iterations):
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]

        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        
        if j % 2000 == 0:            
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix)
                print_sample(sampled_indices, ix_to_char)
                  
            print('\n')

    return parameters

In [76]:
parameters = model(data, ix_to_char, char_to_ix)

Iljjopjoh
Bgwhmllqhhmgufzakodnvdxdsmsgfzkhniwsnfasrxdpyaepdf
Gxxzynvgtnbkrwashbccqfzzbdvidlrsozjgicpieipfkqa
Oansgedqlcqlw
Pfs
Kzwohvsmrnqxnzku
Yvpbdlornbdmfnrntwvnaaodazeicizlqgpncqimsmhxyxgpwv


Pagadadkon
Zauyonasaurus
S
Psaurudgdavnus
Amavoctosaurusabrus
Icelanghlitanoautanesaurus
Saurymnlano


Wingoles
Ydiseonasaurus
Cochcesnzlontodskesaleongostisavoctena
Onazlingps
Lacilaenuv
Ucigevatosescyuameraton
Goontos


Chelimus
Osanrapteis
Hilunomodelnymora
Mentoondsplosoin
Tcondpospmepaxhopcublntotrinibaltojatasnus
Mjtmisaurus
Saurustocongllosaurus


Anachongdun
Mongeanndasaurus
Axngda
Pincadordon
Hugumensaurus
Tankaanongsaurus
Hcheaces


Pasaeatoreshashipeotips
Ynoshyaliyomesaurus
Anfeasaurus
Lomacrosaurustrodenosaurus
Toreosaurus
Hanuoheoeleroqasaurus
Anasaurus


Angassaurus
Siachaakpscraceltalceratops
Hangosaurus
Pisaurus
Actotaritocechagosaurasterotisaurus
Oraphorngong
Angodasaurus


Eropamia
Awakatosaurus
Amcceaus
Ontaloa
Desaloos
Galiadoncuritavsus
Acerikaudangsudona


Acosaurus
Eme